# 네이버 매장 -> 매장에 있는 tag 

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [67]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from tqdm import tqdm
from bs4 import BeautifulSoup

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd

In [68]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
original_df = original_res_df.copy()
original_df = original_df.loc[original_df['검색여부'] == '가능'].reset_index(drop=True)
original_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
original_df['검색어'] = original_df['상세주소'] + " " + original_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
original_df = original_df[column_order]
original_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [55]:
# original_df.to_excel('./data/res_original_df.xlsx')

### tag 데이터 크롤링

In [104]:
start_num = 0 # 할 차례
num = 100
end_num = start_num + num
res_df = original_df[start_num:end_num]
res_df = res_df.reset_index(drop=True)
res_df.head(3)

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"논현로85길 5-13, 아쭈맛나",한식,아쭈맛나,아쭈맛나,서울특별시,강남구,"논현로85길 5-13,",서울특별시 강남구 역삼동 738-17,"서울특별시 강남구 논현로85길 5-13, 지상1층 101호 (역삼동)",203198.556492,444024.178990,가능,영업/정상
1,"언주로 506, 순찌보쌈",한식,순찌보쌈,순찌보쌈,서울특별시,강남구,"언주로 506,",서울특별시 강남구 역삼동 701-1 역삼 아르누보씨티,"서울특별시 강남구 언주로 506, 역삼 아르누보씨티 지상1층 110호 (역삼동)",203716.938086,444579.034009,가능,영업/정상
2,"언주로 506, 덮깨비",한식,덮깨비,덮깨비,서울특별시,강남구,"언주로 506,",서울특별시 강남구 역삼동 701-1 역삼 아르누보씨티,"서울특별시 강남구 언주로 506, 역삼 아르누보씨티 지상1층 109호 (역삼동)",203716.938086,444579.034009,가능,영업/정상


In [105]:
# # 해민
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()

# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())

# 빈 리스트 생성
restaurant_name_list = []
category_name_list = []
price_list = []

for i in tqdm(range(0, len(res_df)), desc='진행 중', position=0, leave=True):
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)  

    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

        for _ in range(4):
        # 스크롤을 위한 JavaScript 코드 실행
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2.5) 

        # HTML 추출
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

        # 여러 식당 정보를 반복하며 리스트에 추가
        for item in items:
            restaurant_name_list.append(name)
            category_name_list.append(item.text.split('"')[1])
            price_list.append(int(''.join(filter(str.isdigit, item.text))))

        data_list = {
            '매장명': restaurant_name_list,
            'tag': category_name_list,
            '인원': price_list
        }
    except :
        data_list = {
            '매장명': restaurant_name_list,
            'tag': '리뷰 10개 미만',
            '인원': ''
        }
        continue

result_df = pd.DataFrame(data_list)
result_df # 100개당 22분 
result_df.to_excel(f'./data/restaurant_tag_data/restaurant_tag_df_{start_num}_{end_num}.xlsx', index=False)

진행 중:  86%|████████▌ | 86/100 [22:10<03:36, 15.47s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.95)


### 리뷰 데이터 크롤링

In [56]:
start_num = 0 # 할 차례
num = 100
end_num = start_num + num
res_df = original_df[start_num:end_num]
new_res_df_test = res_df.reset_index(drop=True)
new_res_df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상
2,"압구정로 343, 수아당",한식,수아당,수아당,서울특별시,강남구,"압구정로 343,",서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",203470.848439,447369.579852,가능,영업/정상
3,"역삼로 204, 알찬한끼",분식,알찬한끼,알찬한끼,서울특별시,강남구,"역삼로 204,",서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",203453.074369,443736.685792,가능,영업/정상
4,"삼성로96길 11, 세계관",한식,세계관,세계관,서울특별시,강남구,"삼성로96길 11,",서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",204952.006872,445195.026116,가능,영업/정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"강남대로118길 28, 카페 바바바",경양식,카페 바바바,카페 바바바(cafe BAVABA),서울특별시,강남구,"강남대로118길 28,",서울특별시 강남구 논현동 183-10,"서울특별시 강남구 강남대로118길 28, 지상1층 (논현동)",202200.375287,444955.562946,가능,영업/정상
96,"논현로 412, 옛날그맛 명동칼국수",한식,옛날그맛 명동칼국수,옛날그맛 명동칼국수,서울특별시,강남구,"논현로 412,",서울특별시 강남구 역삼동 727-14 프라임빌딩,"서울특별시 강남구 논현로 412, 프라임빌딩 지상1층 105호,106호 (역삼동)",203390.390364,443864.765894,가능,영업/정상
97,"강남대로84길 24-4, 뉴욕바",한식,뉴욕바,뉴욕바,서울특별시,강남구,"강남대로84길 24-4,",서울특별시 강남구 역삼동 827-3,"서울특별시 강남구 강남대로84길 24-4, 지하1층 (역삼동)",202635.525941,443839.933605,가능,영업/정상
98,"학동로 343, 후토루",일식,후토루,후토루 강남구청점,서울특별시,강남구,"학동로 343,",서울특별시 강남구 논현동 119 The Pinnacle Gangnam,"서울특별시 강남구 학동로 343, The Pinnacle Gangnam 지하2층 B...",203522.611749,446136.224972,가능,영업/정상


# 음식점 리뷰 크롤링

In [59]:
# 빈 리스트 생성
user_id_list = []
content_list = []
date_list = []  
revisit_list = []
tag_category_name_list = []
restaurant_name_list = []
review_category_name_list = []

# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())


for i in range(0,len(new_res_df_test)):
  
    name = new_res_df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

    except :
        print('끝')


# ---------------- 여기가 수정 -------------------------
    # 현재 페이지 URL 가져오기
    current_url = driver.current_url

    try : 
        # 리뷰 url로 이동 
        modified_url = current_url.replace('/home', '/review/visitor')
        driver.get(modified_url)
    except :
        continue

    # 리뷰 스크래핑 시작
    try:
        # 스크롤
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
            
            # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # 더보기 클릭
            button = driver.find_element_by_class_name('fvwqf')
            button.click()

            # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
            review_buttons = driver.find_elements_by_class_name('xHaT3')  
            for button1 in review_buttons:
                try:
                    button1.click()
                except :
                    pass

            tag_buttons = driver.find_elements_by_css_selector('.gyAGI a.P1zUJ.ZGKcF')  
            for button2 in tag_buttons:
                try:
                    button2.click()
                except :
                    pass
            # ----------------------------------------------
            time.sleep(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            reviews = soup.select('li.YeINN') 

        # 리뷰 크롤링 시작
        for r in reviews: 
            user_id = r.select_one('div.VYGLG')
            content = r.select_one('div.ZZ4OK.IwhtZ')
            try :
                tag = r.select_one('div.gyAGI span.P1zUJ').text
            except :
                tag = ""
            date = r.select('div._7kR3e>span.tzZTd>time')[0]
            revisit = r.select('div._7kR3e>span.tzZTd')[1]
            # revisit = revisit.replace('번째 방문','')

            # 내용이 없을 때
            user_id = user_id.text if user_id else ''
            content = content.text if content else ''
            tag = tag if tag else ''
            date = date.text if date else ''
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            # 리스트에 추가 
            restaurant_name_list.append(name)
            user_id_list.append(user_id)
            content_list.append(content)
            review_category_name_list.append(tag)
            date_list.append(date)
            revisit_list.append(revisit)    

    except Exception as e:
        print(e)

    finally:
        driver.quit()

# 데이터프레임 얹히기
review_data_list = {
    '검색어': restaurant_name_list,
    'user_id': user_id_list,
    'review': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}

# 리뷰 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df


Message: 



MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59966): Max retries exceeded with url: /session/700788d19558705f3115e9c043ef5ed5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE0425E380>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [26]:
review_result_df.to_excel(f'./data/restaurant_review_data/restaurant_review_df_{start_num}_{end_num}.xlsx', index=False)

In [ ]:
# 원본

url = 'https://m.place.naver.com/restaurant/13149660/review/visitor?entry=pll'

# 매장명
place = '강남역_서브웨이'

# 리뷰 스크래핑 시작
try:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    res = driver.get(url)
    driver.implicitly_wait(2)

    # 스크롤
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 
        
        # '더보기' 클릭
        more_button = driver.find_element(By.XPATH, '//div[@class="lfH3O"]/a[@role="button"]')
        more_button.click()

        # 이걸 하니까 적당한 사람 myplace url 생김. 상단 3명 url 가져오라고 할까?
        # try:
        #     while True:
        #         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
        #         time.sleep(0.4)
        # except Exception as e:
        #     print('finish')

    time.sleep(3)
    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    reviews = bs.select('li.YeINN')

    for r in reviews: # 왜 안돌아가?
        user_id = r.select_one('div.VYGLG')
        content = r.select_one('div.ZZ4OK.IwhtZ')
        try :
            tag = r.select_one('div.gyAGI span.P1zUJ').text
        except :
            tag = ""
        date = r.select('div._7kR3e>span.tzZTd>time')[0]
        revisit = r.select('div._7kR3e>span.tzZTd')[1]

        # 아무것도 없을 때
        user_id = user_id.text if user_id else ''
        content = content.text if content else ''
        tag = tag if tag else ''
        date = date.text if date else ''
        revisit = revisit.text if revisit else ''
        time.sleep(0.06)

        # Append data to lists
        user_id_list.append(user_id)
        content_list.append(content)
        review_category_name_list.append(tag)
        date_list.append(date)
        revisit_list.append(revisit)            

except Exception as e:
    print(e)

finally:
    driver.quit()

# Create a DataFrame
review_data_list = {
    #'매장명': restaurant_name_list,
    'user_id': user_id_list,
    'content': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}



# 태그 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df

# test : 한 번에 tag와 review data를 가져오고 싶은데 마음대로 안됨...

In [15]:
# 일부 테스트
res_df_test = res_df.copy()
new_res_df_test = res_df_test.iloc[:5].copy()
new_res_df_test.reset_index(drop=True, inplace=True)
new_res_df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상
2,"압구정로 343, 수아당",한식,수아당,수아당,서울특별시,강남구,"압구정로 343,",서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",203470.848439,447369.579852,가능,영업/정상
3,"역삼로 204, 알찬한끼",분식,알찬한끼,알찬한끼,서울특별시,강남구,"역삼로 204,",서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",203453.074369,443736.685792,가능,영업/정상
4,"삼성로96길 11, 세계관",한식,세계관,세계관,서울특별시,강남구,"삼성로96길 11,",서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",204952.006872,445195.026116,가능,영업/정상


In [21]:
# driver = webdriver.Chrome(ChromeDriverManager().install())

# # 전체 데이터프레임 초기화
# result_df = pd.DataFrame()

# # 리뷰 데이터 프레임 초기화
# review_result_df = pd.DataFrame()

# for i in range(len(new_res_df_test)):
#     name = new_res_df_test['검색어'][i]

#     # 결과 데이터프레임에 추가할 데이터 초기화
#     restaurant_name_list = []
#     category_name_list = []
#     price_list = []

#     try:
#         # 검색 페이지로 이동
#         driver.get('https://map.naver.com/p/search/{}'.format(name))
#         time.sleep(3)

#         # entryIframe이 있는지 확인
#         if driver.find_elements(By.ID, 'entryIframe'):
#             entryIframe = driver.find_element(By.ID, 'entryIframe')
#             driver.switch_to.frame(entryIframe)

#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')

#         # '리뷰' 탭의 href 속성 가져오기
#         review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
#         review_url = 'https://pcmap.place.naver.com' + review_tab_href

#         driver.get(review_url)
#         time.sleep(2.2)

#         for _ in range(4):
#             # 스크롤을 위한 JavaScript 코드 실행
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2)

#         # HTML 추출
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')
#         items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

#         # 여러 식당 정보를 반복하며 리스트에 추가
#         for item in items:
#             restaurant_name_list.append(name)
#             category_name_list.append(item.text.split('"')[1])
#             price_list.append(int(''.join(filter(str.isdigit, item.text))))

#         data_list = {
#             '매장명': restaurant_name_list,
#             'tag': category_name_list,
#             '인원': price_list
#         }

#     except:
#         data_list = {
#             '매장명': restaurant_name_list,
#             'tag': '리뷰 10개 미만',
#             '인원': ''
#         }

#     # 결과 데이터프레임에 추가
#     result_df = pd.concat([result_df, pd.DataFrame(data_list)])

#     # 리뷰 데이터프레임에 추가할 데이터 초기화
#     restaurant_name_list = []
#     user_id_list = []
#     content_list = []
#     review_category_name_list = []
#     date_list = []
#     revisit_list = []

#     try:
#         # 현재 페이지 URL 가져오기
#         current_url = driver.current_url

#         # 리뷰 url로 이동
#         modified_url = current_url.replace('/home', '/review/visitor')
#         driver.get(modified_url)

#         # 리뷰 스크래핑 시작
#         for _ in range(3):
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2)

#             # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
#             button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
#             driver.execute_script("arguments[0].scrollIntoView();", button)
#             button.click()

#             # 더보기 클릭
#             button = driver.find_element_by_class_name('fvwqf')
#             button.click()

#             # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
#             driver.execute_script("arguments[0].scrollIntoView();", button)
#             button.click()

#             # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
#             review_buttons = driver.find_elements_by_class_name('xHaT3')
#             for button1 in review_buttons:
#                 try:
#                     button1.click()
#                 except:
#                     pass

#             tag_buttons = driver.find_elements_by_css_selector('.gyAGI a.P1zUJ.ZGKcF')
#             for button2 in tag_buttons:
#                 try:
#                     button2.click()
#                 except:
#                     pass
#             # ----------------------------------------------
#             time.sleep(3)
#             html = driver.page_source
#             soup = BeautifulSoup(html, 'lxml')
#             reviews = soup.select('li.YeINN')

#         # 리뷰 데이터프레임에 추가할 데이터 초기화
#         restaurant_name_list = []
#         user_id_list = []
#         content_list = []
#         review_category_name_list = []
#         date_list = []
#         revisit_list = []

#         # 리뷰 크롤링 시작
#         for r in reviews:
#             user_id = r.select_one('div.VYGLG')
#             content = r.select_one('div.ZZ4OK.IwhtZ')
#             try:
#                 tag = r.select_one('div.gyAGI span.P1zUJ').text
#             except:
#                 tag = ""
#             date = r.select('div._7kR3e>span.tzZTd>time')[0]
#             revisit = r.select('div._7kR3e>span.tzZTd')[1]

#             # 내용이 없을 때
#             user_id = user_id.text if user_id else ''
#             content = content.text if content else ''
#             tag = tag if tag else ''
#             date = date.text if date else ''
#             revisit = revisit.text if revisit else ''
#             time.sleep(0.06)

#             # 리스트에 추가
#             restaurant_name_list.append(name)
#             user_id_list.append(user_id)
#             content_list.append(content)
#             review_category_name_list.append(tag)
#             date_list.append(date)
#             revisit_list.append(revisit)

#     except Exception as e:
#         print(e)

#     finally:
#         # 리뷰 결과 데이터프레임에 추가
#         review_data_list = {
#             '검색어': restaurant_name_list,
#             'user_id': user_id_list,
#             'review': content_list,
#             'date': date_list,
#             'revisit': revisit_list,
#             'tag': review_category_name_list
#         }
#         review_result_df = pd.concat([review_result_df, pd.DataFrame(review_data_list)])

#         # 드라이버 초기화
#         driver.quit()

# # 최종 결과 출력
# review_result_df.reset_index(drop=True, inplace=True)
# result_df.reset_index(drop=True, inplace=True)
# print(result_df)
# print(review_result_df)


Message: 



HTTPConnectionPool(host='127.0.0.1', port=51292): Max retries exceeded with url: /session/002ee156350cf7f453ca29511233ffcd/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE7ABD2C80>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


HTTPConnectionPool(host='127.0.0.1', port=51292): Max retries exceeded with url: /session/002ee156350cf7f453ca29511233ffcd/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE7ABD2C20>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


HTTPConnectionPool(host='127.0.0.1', port=51292): Max retries exceeded with url: /session/002ee156350cf7f453ca29511233ffcd/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE7CA5BA90>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


HTTPConnectionPool(host='127.0.0.1', port=51292): Max retries exceeded with url: /session/002ee156350cf7f453ca29511233ffcd/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE7ABD3280>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


                매장명       tag  인원
0  강남대로114길 18, 정식당  음식이 맛있어요  11
1  강남대로114길 18, 정식당  재료가 신선해요   9
2  강남대로114길 18, 정식당  가성비가 좋아요   8
3  강남대로114길 18, 정식당      친절해요   6
4  강남대로114길 18, 정식당    양이 많아요   5
Empty DataFrame
Columns: [검색어, user_id, review, date, revisit, tag]
Index: []


In [ ]:
# # 빈 리스트 생성
# user_id_list = []
# content_list = []
# date_list = []  
# revisit_list = []
# tag_category_name_list = []
# restaurant_name_list = []
# review_category_name_list = []
# price_list = []


# # 지선
# driver = webdriver.Chrome(ChromeDriverManager().install())

# # 해민
# driver = webdriver.Chrome()


# # 음식점 이름으로 검색하기 
# for i in range(0,len(new_res_df_test)):
  
#     name = new_res_df_test['검색어'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))
#     time.sleep(3)  


#     try :
#         if driver.find_elements(By.ID,'entryIframe') :
#             entryIframe = driver.find_element(By.ID,'entryIframe')
#             driver.switch_to.frame(entryIframe)
#     except :
#         pass 

#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')


#     # '리뷰' 탭의 href 속성 가져와서 리뷰 탭으로 이동하기 
#     try : 
#         review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
#         review_url = 'https://pcmap.place.naver.com'+review_tab_href
#         driver.get(review_url)
#         time.sleep(3)  

#         # 스크롤
#         for _ in range(3):
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2) 


#         # tag 추출 
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')
#         items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

#         # 식당 tag를 리스트에 추가
#         for item in items:
#             restaurant_name_list.append(name)
#             tag_category_name_list.append(item.text.split('"')[1])
#             price_list.append(int(''.join(filter(str.isdigit, item.text))))

#         tag_data_list = {
#             '매장명': restaurant_name_list,
#             'tag': tag_category_name_list,
#             '인원': price_list
#         }

#     #     # 여기가 안 먹혀요. 수정이 필요합니다. 
#     except :
#         tag_data_list = {
#             '매장명': restaurant_name_list,
#             'tag': '리뷰 10개 미만',
#             '인원': ''
#         }
#         continue



# # # 리뷰 스크래핑 시작 -> 여기가 합쳐놓으니 안먹혀요
# try:
#     # driver = webdriver.Chrome(ChromeDriverManager().install())
#     # url = 'https://m.place.naver.com/restaurant/1159180795/review/visitor'
#     # res = driver.get(url)
#     # driver.implicitly_wait(2)

#     # 스크롤
#     for _ in range(4):
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2) 

#         # '더보기' 클릭
#         more_button = driver.find_element(By.XPATH, '//div[@class="lfH3O"]/a[@role="button"]')
#         more_button.click()
    
    
#     driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
#     time.sleep(0.4)

#     html = driver.page_source
#     bs = BeautifulSoup(html, 'lxml')
#     reviews = bs.select('li.YeINN')

#     for r in reviews:
#         user_id = r.select_one('div.VYGLG')
#         content = r.select_one('div.ZZ4OK.IwhtZ')
#         tag = r.select_one('div.gyAGI span.P1zUJ').text
#         date = r.select('div._7kR3e>span.tzZTd>time')[0]
#         revisit = r.select('div._7kR3e>span.tzZTd')[1]

#         # 아무것도 없을 때
#         user_id = user_id.text if user_id else ''
#         content = content.text if content else ''
#         tag = tag if tag else ''
#         date = date.text if date else ''
#         revisit = revisit.text if revisit else ''
#         time.sleep(0.06)

#         # Append data to lists
#         user_id_list.append(user_id)
#         content_list.append(content)
#         review_category_name_list.append(tag)
#         date_list.append(date)
#         revisit_list.append(revisit)

# # Handle exceptions
# except Exception as e:
#     print(e)

# # # Finally, close the WebDriver
# # finally:
# #     driver.quit()

# # Create a DataFrame
# review_data_list = {
#     #'매장명': restaurant_name_list,
#     'user_id': user_id_list,
#     'content': content_list,
#     'date': date_list,
#     'revisit': revisit_list,
#     'tag': review_category_name_list
# }

# # 태그 데이터 프레임 완성
# tag_result_df = pd.DataFrame(tag_data_list)
# review_result_df = pd.DataFrame(review_data_list)



Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="lfH3O"]/a[@role="button"]"}
  (Session info: chrome=122.0.6261.57)



In [ ]:
tag_result_df.tail()

,매장명,tag,인원
5,"양재대로33길 24, 브레드_히어_",빵이 맛있어요,13
6,"양재대로33길 24, 브레드_히어_",건강한 맛이에요,6
7,"양재대로33길 24, 브레드_히어_",친절해요,5
8,"양재대로33길 24, 브레드_히어_",매장이 청결해요,5
9,"양재대로33길 24, 브레드_히어_",아늑해요,3


In [ ]:
review_result_df.tail()

,user_id,content,date,revisit,tag


In [ ]:
# review_result_df.to_excel('.data/음식점_리뷰데이터.xlsx')
# tag_result_df.to_excel('.data/음식점_태그데이터.xlsx')